In [1]:
import inspect

In [ ]:
python类型注解
函数注解(function annotations):
只是对函数参数做辅助说明，非强制性定义参数类型
也不会对参数进行检查
变量注解：
i:int=3

In [10]:
def add(a:int, b:int):
    pass

In [11]:
print(add.__name__, add.__doc__, add.__dict__, add.__annotations__)

add None {} {'a': <class 'int'>, 'b': <class 'int'>}


In [12]:
add(1,2) # 假设传入了位置参数无法和字典中的声明对比？？？？？

In [ ]:
参数检查的过程：
函数作为参数传入到检查函数中
检查参数拿到函数传入的实际参数，与形参声明对比

In [2]:
# inspect模块
import inspect, pathlib
inspect.isfunction(add)
inspect.ismethod(pathlib.Path().absolute) # 是类的方法
inspect.isgenerator((i for i in range(2))) # 是生成器表达式构成的生成器
inspect.isgeneratorfunction(add) #  函数中有yield
inspect.isgeneratorfunction((i for i in range(2))) # 不是生成器函数
inspect.isclass(add)
inspect.ismodule(inspect)
inspect.isbuiltin(print) 
inspect.isbuiltin(int) # 不是内建函数，是被导入的


False

In [6]:
def add(x:int, y:int=2, *args, m:int, n:int, **kwargs) -> int:
    return x + y + m +n

In [7]:
# 参数是可调用对象，包含定义变量的类型和返回值的类型
inspect.signature(add)

<Signature (x:int, y:int=2, *args, m:int, n:int, **kwargs) -> int>

In [8]:
# 返回OrderedDict类型，键值对，键为函数中定义的变量名，值储存了变量的参数是一个
# Parameter对象，对象中的属性有name(参数名)，annotation(参数注释，如果函数定义时没有写：int,则显示为empty)
# default(参数的缺省值，如果定义函数时没给，则显示empty)
# empty()
# 值中的参数有
inspect.signature(add).parameters

mappingproxy(OrderedDict([('x', <Parameter "x:int">), ('y', <Parameter "y:int=2">), ('args', <Parameter "*args">), ('m', <Parameter "m:int">), ('n', <Parameter "n:int">), ('kwargs', <Parameter "**kwargs">)]))

In [12]:
x_parameter = inspect.signature(add).parameters['x']

print(x_parameter.annotation)
print(x_parameter.name)
print(x_parameter.default)
print(x_parameter.empty)
print(x_parameter.kind)

<class 'int'>
x
<class 'inspect._empty'>
<class 'inspect._empty'>
POSITIONAL_OR_KEYWORD


In [14]:
y_parameter = inspect.signature(add).parameters['y']

In [15]:
print(x_parameter.annotation)
print(x_parameter.name)
print(y_parameter.default)
# 用来标记 default或annotation空值，无论是否为空都会打印<class 'inspect._empty'>
print(y_parameter.empty)
print(y_parameter.kind)

<class 'int'>
x
2
<class 'inspect._empty'>
POSITIONAL_OR_KEYWORD


In [23]:
# add中定义的参数有x, y, *args, m, n, **kwargs
# kind有4种类型，POSITIONAL_OR_KEYWORD, VAR_POSITIONAL
# KEYWORD_ONLY, VAR_KEYWORD, 后面对应相应的编号
inspect.signature(add).parameters['kwargs'].kind

<_ParameterKind.VAR_KEYWORD: 4>

In [ ]:
# 例子：判断用户输入的参数是否否符合要求

In [26]:
from functools import wraps
def logger(fn):
    @wraps(fn)  # wrapper = param(fn)(wrapper)
    def wrapper(*args, **kwargs):
        params = inspect.signature(wrapper).parameters
        values = list(params.values())
        flag = False
        for i, arg in enumerate(args):
            value = values[i]
            if isinstance(arg, value.annotation):
                print(arg, 'right')
            else:
                print(arg, 'rong')
                flag = True
                break
        if not flag:
            for k, kwarg in kwargs.items():
                value = params[k]
                if isinstance(kwarg, value.annotation):
                    print(kwargs, 'right')
                else:
                    print(kwargs, 'roung')
                    flag = True
                    break
            
        ret = fn(*args,**kwargs)
        return ret
    return wrapper

@logger
def add(x:int=1, y:int=2) -> int:
    return x + y

In [30]:
add('a','b')

a rong


'ab'

In [31]:
from functools import wraps
def logger(fn):
    @wraps(fn)  # wrapper = param(fn)(wrapper)
    def wrapper(*args, **kwargs):
        params = inspect.signature(wrapper).parameters
        values = list(params.values())
        flag = False
        for i, arg in enumerate(args):
            value = values[i]
            if isinstance(arg, value.annotation):
                print(arg, 'right')
            elif value.annotation is value.empty:
                print('any type')
            else:
                print(arg, 'wrong')
                flag = True
            
        if not flag:
            for k, kwarg in kwargs.items():
                value = params[k]
                if isinstance(kwarg, value.annotation):
                    print(kwargs, 'right')
                elif value.annotation is value.empty:
                    print('any type')
                else:
                    print(kwargs, 'wrong')
                    flag = True
                    
            
        ret = fn(*args,**kwargs)
        return ret
    return wrapper


# 定义时x或y没给类型，传入参数调用函数时会提示any type
# 定义时x或y给了类型，传入参数调用函数时会提示right 或者 wrong
@logger
def add(x, y:int=2) -> int:
    return x + y

In [34]:
add('a', 'b')

any type
b rong


'ab'

In [ ]:
def param(fn):
    def _param(wrapper):
        params = inspect.signature(wrapper).parameters
        for i, (k,v) in enumerate(params.items()):
            print(i, k, v.name, v.kind, v.default, v.annotation)
        #ret = fn(x, y, *args, m, n, **kwargs)
        return wrapper
    return _param

def logger(fn):
    @param(fn)  # wrapper = param(fn)(wrapper)
    def wrapper(x, y, *args, m, n, **kwargs):
        ret = fn(x, y, *args, m, n, **kwargs)
        return ret
    return wrapper

In [4]:
def add(x, y:int=7, *args, m, n:int=10, **kwargs) -> int:
    return x + y
sig = inspect.signature(add)
print(sig)
params = sig.parameters
print(params)
for i, (k,v) in enumerate(params.items()):
    print(i, k, v.name, v.kind, v.default, v.annotation)

(x, y:int=7, *args, m, n:int=10, **kwargs) -> int
OrderedDict([('x', <Parameter "x">), ('y', <Parameter "y:int=7">), ('args', <Parameter "*args">), ('m', <Parameter "m">), ('n', <Parameter "n:int=10">), ('kwargs', <Parameter "**kwargs">)])
0 x x POSITIONAL_OR_KEYWORD <class 'inspect._empty'> <class 'inspect._empty'>
1 y y POSITIONAL_OR_KEYWORD 7 <class 'int'>
2 args args VAR_POSITIONAL <class 'inspect._empty'> <class 'inspect._empty'>
3 m m KEYWORD_ONLY <class 'inspect._empty'> <class 'inspect._empty'>
4 n n KEYWORD_ONLY 10 <class 'int'>
5 kwargs kwargs VAR_KEYWORD <class 'inspect._empty'> <class 'inspect._empty'>


In [38]:
# functools 模块
from functools import reduce
#reduce(function, sequence[,initial]) -> value

In [40]:
# 求和
nums = range(10)
print(reduce(lambda val, x: val + x, nums))

45


In [ ]:
# 返回一个新函数，对原函数function进行封装，把function的部分功能固定下来了
# partial(function, parameter)

In [54]:
# 例子：
import functools
def add(x, y, *args) -> int:
    print(args)
    return x + y 
newadd = functools.partial(add, 1,2,3,4)
newadd(5) # 1,2已经被固定住了，5和3，4一起被收入args中
#newadd(y=10, x=10) # 错误写法因为x 和 y已经被固定住了
inspect.signature(newadd) # <Signature (*args) -> int> newadd只有args参数

(3, 4, 5)


<Signature (*args) -> int>

In [57]:
# partial函数的本质
def partial(func, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = keywords.copy()
        newkeywords.update(fkeywords)   
        return func(*(args + fargs), **newkeywords)       
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc

def add(x, y):
    return x + y

foo = partial(add, 1, 2)
foo()

3

In [ ]:
lru_cache(maxsize=128, typed=False)
lru(Least-recently-used) 最近最少使用
maxsize = None 禁止lru功能，缓存可以无限制增长，是2的幂时lru功能执行的最好
typed = True 不同类型的参数将单独缓存f(3),f(3.0),将对应不同的结果

In [60]:
import time
@functools.lru_cache()
def add(a, b):
    time.sleep(2)
    return a + b
add(1, 2) # 慢
add(1, 2) # 快
add(1, b=2) # 慢
add(a=1, b=2) # 慢 
缓存的机制？ 内部的键值对是如何构建的？

3

In [68]:
functools._make_key??

In [ ]:
def _make_key(args, kwds, typed,
             kwd_mark = (object(),),
             fasttypes = {int, str, frozenset, type(None)},
             tuple=tuple, type=type, len=len):
    """Make a cache key from optionally typed positional and keyword arguments

    The key is constructed in a way that is flat as possible rather than
    as a nested structure that would take more memory.

    If there is only a single argument and its data type is known to cache
    its hash value, then that argument is returned without a wrapper.  This
    saves space and improves lookup speed.

    """
    key = args
    if kwds:
        key += kwd_mark
        for item in kwds.items():
            key += item
    if typed:
        key += tuple(type(v) for v in args)
        if kwds:
            key += tuple(type(v) for v in kwds.values())
    elif len(key) == 1 and type(key[0]) in fasttypes:
        return key[0]
    return _HashedSeq(key)

In [66]:
@functools.lru_cache()
def fib(n):
    return 1 if n < 3 else fib(n-1) + fib(n-2)
print(fib(35))
#print([fib(i+1) for i in range(1, 36)])

9227465


In [ ]:
lru_cache装饰器的特点
优点：同样的函数参数一定得到同样的结果
缺点：不支持缓存过期，key无法过期，失效
    不支持清楚操作
    不支持分布式，是一个单机的缓存
